In [ ]:
import pandas as pd
import os
import numpy as np
#nibabel allows us to load up the freesurfer file types
import nibabel as nib
import HiguchiFractalDimension as hfd
import matplotlib.pyplot as plt
import scipy as sp
from scipy import stats



In [ ]:

LUT = pd.read_csv("LUT.tsv", sep="\t")
LUT = LUT.drop(columns= ['R','G','B','A'])
region_labels= LUT.drop(columns= 'No')
LUT= LUT.set_index('No')
LUT.index.astype(int)
LUT.head()


In [ ]:
print(LUT.at[2035, 'Name'])
#just testing 

In [ ]:
regions_array = region_labels['Name'].to_list()
regions_array = np.insert(regions_array, 0, 'lh-ribbon')
regions_array = np.insert(regions_array, 0, 'rh-ribbon')
print(regions_array)

In [ ]:
#clean up data
data= pd.read_csv("*enter dataset*")
data= data.drop(columns= ['IS IN BOTH','amndem', 'STANDARD ID', 'racecode','MCIAMEM', 'MCIAPLUS'])
data.insert(2, 'GROUP', None)
for sub in data.index:
    if data.at[sub, 'NORMCOG']==1:
        data.at[sub, 'GROUP']= 'CN'

    elif data.at[sub, 'alzdis']==1:
        data.at[sub, 'GROUP']= 'AD'



In [ ]:
vol_stats = ['BEST FD', 'R2' , 'IDEAL RANGE INC. ']
#see? a multiindex table for organisation
vol_stats = pd.MultiIndex.from_product([regions_array, vol_stats])
vol_stats = pd.DataFrame(columns=vol_stats, index= data['SUB_ID'])
vol_stats.head()

In [ ]:
vol_stats.head()

In [ ]:
base_path = "*ENTER BASE PATH*"

In [ ]:
#calculating fractal dimension of a brain region using volumetric files

vol_data = data.copy()
def voxel_box_counting(box_size):
    x, y, z = ROI.shape
    sx= sy= sz = box_size
    reshaped_ROI = ROI.reshape(
        x // sx, sx,
        y // sy, sy,
        z // sz, sz

    ).sum(axis=(1, 3, 5))   #reshape into a new array of box size *box size * box size dimensions
    num_boxes= int((np.sum(reshaped_ROI > 0))) #count the number of new voxels that fill the shape
    binary = np.any(reshaped_ROI)

    return num_boxes

for ID in vol_data['SUB_ID']:


    MR_ID= vol_data.loc[vol_data['SUB_ID']==ID, 'MR_session'].iloc[0].upper()
    segment = os.path.join(base_path, MR_ID, "mri","aparc+aseg.mgz")
    segment = nib.load(segment)
    segment = segment.get_fdata()
    for LABEL in LUT.index:
        ROI_labels = [LABEL]
        #corpus callosum is a combination of labels
        if ROI_labels == [0]:
            ROI_labels = [251,252,253,254,255]
        ROI_NAME= LUT.at[LABEL, 'Name']
        ROI = np.isin(segment, ROI_labels).astype(np.uint8)
        #the result is shown as a 3d array of the region which we can now use 
        if ROI_NAME =='CC_Posterior':
            continue
        #skip a region if empty
        skipped = 0
        # counting regular voxels that contain data, to ensure it contains a volume
        voxels= np.sum(ROI>0)
        if voxels == 0:
            print(ROI_NAME, "Region is empty, skipping to next region", ID)
            skipped += 1
            continue

        #cropping the ROI
        present_voxels = np.argwhere(ROI)
        ROI = ROI[
            present_voxels[:, 0].min()-1: present_voxels[:, 0].max() + 1,
            present_voxels[:, 1].min()-1: present_voxels[:, 1].max() + 1,
            present_voxels[:, 2].min()-1: present_voxels[:, 2].max() + 1,
        ]
        print(ROI.shape)
        #expanding the ROI to make it a cube
        sx, sy, sz = ROI.shape
        maximum = max(sx, sy, sz)


        ROI= np.pad(
            ROI,
            pad_width=((0, 240 - sx), (0, 240 - sy), (0, 240 - sz)),
            mode='constant',
            constant_values=0)
        padded_shape=240


        y = np.zeros(40)
        x=np.zeros(40)
        idx = 0
        for i in range(padded_shape):
            box_size= i+1
            if padded_shape%box_size==0 and box_size<=maximum:
                box_size=int(box_size)
                num_box= voxel_box_counting(box_size)
                y[idx]= num_box
                x[idx]= box_size
                idx += 1
        x = x.astype(int)
        y = y.astype(int)   
        y=y[0:idx]
        x=x[0:idx]

        log_x= np.log(x)
        log_y=np.log(y)


        widest_range = 0 
        best_fd = 0
        best_r2 = 0
        min_j = 0
        max_k = 0
        for j in range(1, idx):
            for k in range(1, idx):
                if k > j:
                    rf_log_x = log_x[j:k+1]
                    rf_log_y = log_y[j:k+1]
                    slope, intercept, r_value, p_value, std_err = stats.linregress(rf_log_x, rf_log_y)
                    r2 = r_value**2
                    fd = -slope
                    if 0.995 < r2 < 1: 
                        if k - j >= widest_range and k <= idx: 
                            widest_range = k - j
                            min_j = j
                            max_k = k
                            best_fd = fd
                            best_r2 = r2    

        if max_k > 0 and min_j >= 0:
            print("Fractal Dimension is: ", best_fd)
            print("its R-squared value is: ", best_r2)
            print("ideal range is from box sizes: ", x[min_j] , " to ", x[max_k])
            vol_stats.loc[ID, (ROI_NAME, 'R2')] = best_r2
            vol_stats.loc[ID, (ROI_NAME, 'BEST FD')] = best_fd
            vol_stats.loc[ID, (ROI_NAME, 'IDEAL RANGE INC. ')] = (str(x[min_j]) + ":" + str(x[max_k]))
        else:
            print(ROI_NAME, "No valid regression range found for", ID)
        print(ROI_NAME, "done, for ", ID)
        print(skipped, "regions were skipped due to being empty.")
        
print("All done!") 


In [ ]:
vol_stats.head()

In [ ]:
data.head()

In [ ]:
#LH of cortical ribbon 
lh_ribbon_data = data.copy()
for ID in data['SUB_ID']:

    MR_ID= lh_ribbon_data.loc[lh_ribbon_data['SUB_ID']==ID, 'MR_session'].iloc[0].upper()
    segment = os.path.join(base_path, MR_ID,"mri", "lh.ribbon.mgz")
    ROI = nib.load(segment)
    ROI = ROI.get_fdata()
    #the result is shown as a 3d array of the region which we can now use 
    ROI_NAME= 'lh-ribbon'

    #padding 
    sx, sy, sz = ROI.shape
    maximum = max(sx, sy, sz)
    ROI= np.pad(
        ROI,
        pad_width=((0, 480 - sx), (0, 480 - sy), (0, 480 - sz)),
        mode='constant',
        constant_values=0)
    padded_shape=480

    y = np.zeros(40)
    x=np.zeros(40)
    idx = 0
    for i in range(padded_shape):
        box_size= i+1
        if padded_shape%box_size==0 and box_size<=maximum:
            box_size=int(box_size)
            num_box= voxel_box_counting(box_size)
            y[idx]= num_box
            x[idx]= box_size
            idx += 1
    x = x.astype(int)
    y = y.astype(int)   
    y=y[0:idx]
    x=x[0:idx]

    log_x= np.log(x)
    log_y=np.log(y)

    widest_range = 0 
    best_fd = 0
    best_r2 = 0
    min_j = 0
    max_k = 0
    for j in range(1, idx):
        for k in range(1, idx):
            if k > j:
                rf_log_x = log_x[j:k+1]
                rf_log_y = log_y[j:k+1]
                slope, intercept, r_value, p_value, std_err = stats.linregress(rf_log_x, rf_log_y)
                r2 = r_value**2
                fd = -slope
                if 0.995 < r2 < 1: 
                    if k - j >= widest_range and k <= idx: 
                        widest_range = k - j
                        min_j = j
                        max_k = k
                        best_fd = fd
                        best_r2 = r2    

    if max_k > 0 and min_j >= 0:
        print("Fractal Dimension is: ", best_fd)
        print("its R-squared value is: ", best_r2)
        print("ideal range is from box sizes: ", x[min_j] , " to ", x[max_k])
        vol_stats.loc[ID, (ROI_NAME, 'R2')] = best_r2
        vol_stats.loc[ID, (ROI_NAME, 'BEST FD')] = best_fd
        vol_stats.loc[ID, (ROI_NAME, 'IDEAL RANGE INC. ')] = (str(x[min_j]) + ":" + str(x[max_k]))
    else:
        print(ROI_NAME, "No valid regression range found for", ID)
    print(ROI_NAME, "done, for ", ID)
    
print("All done!") 


In [ ]:
vol_stats.head()


In [ ]:
#RH of cortical ribbon 
rh_ribbon_data = data.copy()
for ID in data['SUB_ID']:

    MR_ID= rh_ribbon_data.loc[rh_ribbon_data['SUB_ID']==ID, 'MR_session'].iloc[0].upper()
    segment = os.path.join(base_path, MR_ID,"mri", "rh.ribbon.mgz")
    ROI = nib.load(segment)
    ROI = ROI.get_fdata()
    #the result is shown as a 3d array of the region which we can now use 
    ROI_NAME= 'rh-ribbon'

    #padding 
    sx, sy, sz = ROI.shape
    maximum = max(sx, sy, sz)
    ROI= np.pad(
        ROI,
        pad_width=((0, 480 - sx), (0, 480 - sy), (0, 480 - sz)),
        mode='constant',
        constant_values=0)
    padded_shape=480

    y = np.zeros(40)
    x=np.zeros(40)
    idx = 0
    for i in range(padded_shape):
        box_size= i+1
        if padded_shape%box_size==0 and box_size<=maximum:
            box_size=int(box_size)
            num_box= voxel_box_counting(box_size)
            y[idx]= num_box
            x[idx]= box_size
            idx += 1
    x = x.astype(int)
    y = y.astype(int)   
    y=y[0:idx]
    x=x[0:idx]

    log_x= np.log(x)
    log_y=np.log(y)


    widest_range = 0 
    best_fd = 0
    best_r2 = 0
    min_j = 0
    max_k = 0
    for j in range(1, idx):
        for k in range(1, idx):
            if k > j:
                rf_log_x = log_x[j:k+1]
                rf_log_y = log_y[j:k+1]
                slope, intercept, r_value, p_value, std_err = stats.linregress(rf_log_x, rf_log_y)
                r2 = r_value**2
                fd = -slope
                if 0.995 < r2 < 1: 
                    if k - j >= widest_range and k <= idx: 
                        widest_range = k - j
                        min_j = j
                        max_k = k
                        best_fd = fd
                        best_r2 = r2    

    if max_k > 0 and min_j >= 0:
        print("Fractal Dimension is: ", best_fd)
        print("its R-squared value is: ", best_r2)
        print("ideal range is from box sizes: ", x[min_j] , " to ", x[max_k])
        vol_stats.loc[ID, (ROI_NAME, 'R2')] = best_r2
        vol_stats.loc[ID, (ROI_NAME, 'BEST FD')] = best_fd
        vol_stats.loc[ID, (ROI_NAME, 'IDEAL RANGE INC. ')] = (str(x[min_j]) + ":" + str(x[max_k]))
    else:
        print(ROI_NAME, "No valid regression range found for", ID)
    print(ROI_NAME, "done, for ", ID)
    
print("All done!") 

In [ ]:
vol_stats.to_csv('results/vol_stats.csv')


In [ ]:
surf_labels= ['unknown', 'bankssts', 'caudalanteriorcingulate', 'caudalmiddlefrontal', 'corpuscallosum', 'cuneus', 'entorhinal', 'fusiform', 'inferiorparietal', 'inferiortemporal', 'isthmuscingulate', 'lateraloccipital', 'lateralorbitofrontal', 'lingual', 'medialorbitofrontal', 'middletemporal', 'parahippocampal', 'paracentral', 'parsopercularis', 'parsorbitalis', 'parstriangularis', 'pericalcarine', 'postcentral', 'posteriorcingulate', 'precentral', 'precuneus', 'rostralanteriorcingulate', 'rostralmiddlefrontal', 'superiorfrontal', 'superiorparietal', 'superiortemporal', 'supramarginal', 'frontalpole', 'temporalpole', 'transversetemporal', 'insula', 'lhthickness', 'rhthickness']
lh_surf_stats = ['BEST FD', 'R2' , 'IDEAL RANGE INC. ']
lh_surf_stats = pd.MultiIndex.from_product([surf_labels, lh_surf_stats])
lh_surf_stats = pd.DataFrame(columns=lh_surf_stats, index= data['SUB_ID'])
lh_surf_stats.head()

In [ ]:

def surf_box_counting(box_size):
    x_pos= np.arange(x_min, x_max, box_size)
    y_pos = np.arange(y_min, y_max, box_size)
    z_pos= np.arange(z_min, z_max, box_size)
    counter=0
    for x in x_pos:
        for y in y_pos:
            for z in z_pos:
                cube = ((ROI_vertices[:,0]>= x) & (ROI_vertices[:,0]< x+box_size) & (ROI_vertices[:,1]>= y) & (ROI_vertices[:,1]< y+box_size) & (ROI_vertices[:,2]>= z) & (ROI_vertices [:,2]< z+box_size))
                if np.any(cube):
                    counter+=1


    return counter

#pial surface-based FD for LH cortical regions
for ID in lh_surf_stats.index:
    MR_ID= data.loc[data['SUB_ID']==ID, 'MR_session'].iloc[0].upper()
    segment = os.path.join(base_path, MR_ID,"surf", "lh.pial")
    vertices, faces = nib.freesurfer.read_geometry(segment)

    annot_seg = os.path.join(base_path, MR_ID,"label","lh.aparc.annot")
    annot = nib.freesurfer.io.read_annot(annot_seg,orig_ids = False)
    #axis 0 shows the labels no.
    #axis 1 shows the coordinates
    #axis 2 shows the label name (str)

    label= annot[0]
    ctab= annot[1]
    name = (annot[2])
    #the name array needs decoding
    for i in range(len(name)):
        name[i]= name[i].decode(encoding="utf-8", errors="strict")
        

    for ROI_label in range(1,len(name)):



        ROI_NAME= name[ROI_label]
        ROI= np.where(label==ROI_label)[0]
        ROI_vertices=vertices[ROI]    


        counter= 0
        for i in ROI_vertices:
            counter += 1

        if counter == 0:
            print("Region is empty, skipping to next region", ID)
            continue


        x_min = np.min(ROI_vertices[:,0])
        x_max = np.max(ROI_vertices[:,0])
        y_min = np.min(ROI_vertices[:,1])
        y_max = np.max(ROI_vertices[:,1])
        z_min = np.min(ROI_vertices[:,2])
        z_max = np.max(ROI_vertices[:,2])

        x= np.zeros(18)
        y= np.zeros(18)
        idx=0
        for i in range(2,240):
            if 240 % i == 0:
                number_of_boxes = surf_box_counting(i)
                box_size = i
                # x is the x axis of our graph, representing box size
                x[idx] = int(box_size)
                y[idx]= int(number_of_boxes)
                idx += 1
        x = x.astype(int)
        y = y.astype(int)   
        y=y[0:idx]
        x=x[0:idx]



        log_x= np.log(x)
        log_y=np.log(y)

        widest_range = 0 
        best_fd = 0
        best_r2 = 0
        min_j = 0
        max_k = 0
        for j in range(1, idx):
            for k in range(1, idx):
                if k > j:
                    rf_log_x = log_x[j:k+1]
                    rf_log_y = log_y[j:k+1]
                    slope, intercept, r_value, p_value, std_err = stats.linregress(rf_log_x, rf_log_y)
                    r2 = r_value**2
                    fd = -slope
                    if 0.995 < r2 < 1: 
                        if k - j >= widest_range and k <= idx: 
                            widest_range = k - j
                            min_j = j
                            max_k = k
                            best_fd = fd
                            best_r2 = r2    

        if max_k > 0 and min_j >= 0:
            print("Fractal Dimension is: ", best_fd)
            print("its R-squared value is: ", best_r2)
            print("ideal range is from box sizes: ", x[min_j] , " to ", x[max_k])
            lh_surf_stats.loc[ID, (ROI_NAME, 'R2')] = best_r2
            lh_surf_stats.loc[ID, (ROI_NAME, 'BEST FD')] = best_fd
            lh_surf_stats.loc[ID, (ROI_NAME, 'IDEAL RANGE INC. ')] = (str(x[min_j]) + ":" + str(x[max_k]))
        else:
            print(ROI_NAME, "No valid regression range found for", ID)
        print(ROI_NAME, "done, for ", ID)
        
print("All done!") 

In [ ]:
lh_surf_stats.head()

In [ ]:
lh_surf_stats.to_csv('results/lh_surf_stats.csv')

In [ ]:
rh_surf_stats = ['BEST FD', 'R2', 'IDEAL RANGE INC. ']
rh_surf_stats = pd.MultiIndex.from_product([surf_labels, rh_surf_stats])
rh_surf_stats = pd.DataFrame(columns=rh_surf_stats, index= data['SUB_ID'])
rh_surf_stats.head()

In [ ]:
#pial surface-based FD for RH cortical regions
for ID in rh_surf_stats.index:
    MR_ID= data.loc[data['SUB_ID']==ID, 'MR_session'].iloc[0].upper()
    segment = os.path.join(base_path, MR_ID,"surf", "rh.pial")
    vertices, faces = nib.freesurfer.read_geometry(segment)

    annot_seg = os.path.join(base_path, MR_ID,"label","rh.aparc.annot")
    annot = nib.freesurfer.io.read_annot(annot_seg,orig_ids = False)
    #axis 0 shows the labels no.
    #axis 1 shows the coordinates
    #axis 2 shows the label name (str)

    label= annot[0]
    ctab= annot[1]
    name = (annot[2])
    #the name array needs decoding
    for i in range(len(name)):
        name[i]= name[i].decode(encoding="utf-8", errors="strict")
        

    for ROI_label in range(1,len(name)):



        ROI_NAME= name[ROI_label]
        ROI= np.where(label==ROI_label)[0]
        ROI_vertices=vertices[ROI]    


        counter= 0
        for i in ROI_vertices:
            counter += 1

        if counter == 0:
            print("Region is empty, skipping to next region", ID)
            continue


        x_min = np.min(ROI_vertices[:,0])
        x_max = np.max(ROI_vertices[:,0])
        y_min = np.min(ROI_vertices[:,1])
        y_max = np.max(ROI_vertices[:,1])
        z_min = np.min(ROI_vertices[:,2])
        z_max = np.max(ROI_vertices[:,2])

        x= np.zeros(18)
        y= np.zeros(18)
        idx=0
        for i in range(2,240):
            if 240 % i == 0:
                number_of_boxes = surf_box_counting(i)
                box_size = i
                # x is the x axis of our graph, representing box size
                x[idx] = int(box_size)
                y[idx]= int(number_of_boxes)
                idx += 1
        x = x.astype(int)
        y = y.astype(int)   
        y=y[0:idx]
        x=x[0:idx]



        log_x= np.log(x)
        log_y=np.log(y)
        widest_range = 0 
        best_fd = 0
        best_r2= 0



        fd_array = np.array([0])
        for j in range(0,idx+1):
            for k in range(1,idx+1):
                if k>j:
                    rf_log_x = log_x[j:k+1]
                    rf_log_y = log_y[j:k+1]
                    m, c = np.polyfit(rf_log_x, rf_log_y, 1)
                    fd = -m
                    correlation_coefficient = np.corrcoef(rf_log_x, rf_log_y)[0,1]
                    r2 = correlation_coefficient**2
                    if  0.995 < r2 < 1:
                        if  k-j >= widest_range and k<idx:
                            widest_range = k - j
                            min_j = j
                            max_k = k
                            best_fd = fd
                            best_r2 = r2    
                            fd_array = np.append(fd_array, fd)

        if max_k > 0 and min_j >= 0:
            print("Fractal Dimension is: ", best_fd)
            print("its R-squared value is: ", best_r2)
            print("ideal range is from box sizes: ", x[min_j] , " to ", x[max_k])
            rh_surf_stats.loc[ID, (ROI_NAME, 'R2')] = best_r2
            rh_surf_stats.loc[ID, (ROI_NAME, 'BEST FD')] = best_fd
            rh_surf_stats.loc[ID, (ROI_NAME, 'IDEAL RANGE INC. ')] = (str(x[min_j]) + ":" + str(x[max_k]))
        else:
            print(ROI_NAME, "No valid regression range found for", ID)
        print(ROI_NAME, "done, for ", ID)
print("All done!")

In [ ]:
rh_surf_stats.head()

In [ ]:
rh_surf_stats.to_csv('results/rh_surf_stats.csv')

In [ ]:
#cortical thickness fd with imported Higuchi method

#lh thickness
# The next step is to determine appropriate k_max, which I have not done yet. This output is based on a default k_max, and may be incorrect
for ID in lh_surf_stats.index:
    MR_ID= data.loc[data['SUB_ID']==ID, 'MR_session'].item().upper()
    segment = os.path.join(base_path, MR_ID,"surf","lh.thickness")
    segment = nib.freesurfer.io.read_morph_data(segment)
    k, Lk = hfd.curve_length(segment)
    hfd_value = hfd.lin_fit_hfd(k,Lk)
    lh_surf_stats.loc[ID, ('lhthickness', 'BEST FD')] = hfd_value
    print ("HFD of lh-thickness:" ,hfd_value)
    print('done for ', ID, "lh-thickness")

#rh thickness
for ID in rh_surf_stats.index:
    MR_ID= data.loc[data['SUB_ID']==ID, 'MR_session'].item().upper()
    segment = os.path.join(base_path, MR_ID,"surf","rh.thickness")
    segment = nib.freesurfer.io.read_morph_data(segment)
    #calculate FD based on linear fitted k values to get k_max
    k, Lk = hfd.curve_length(segment)
    hfd_value = hfd.lin_fit_hfd(k,Lk)
    rh_surf_stats.loc[ID, ('rhthickness', 'BEST FD')] = hfd_value
    print("HFD of rh-thickness:", hfd_value)
    print('done for ', ID, "rh-thickness")



In [ ]:
rh_surf_stats.to_csv('results/rh_surf_stats.csv')
lh_surf_stats.to_csv('results/lh_surf_stats.csv')